# Deploy Mnist model with Triton

## Prerequisites

 * A kubernetes cluster with kubectl configured
%%writefile rclone.conf
[s3]
type = s3
provider = minio
env_auth = false
access_key_id = minioadmin
secret_access_key = minioadmin
endpoint = http://localhost:8090 * rclone
 * curl

## Setup Seldon Core

Use the setup notebook to [Setup Cluster](https://docs.seldon.io/projects/seldon-core/en/latest/examples/seldon_core_setup.html#Setup-Cluster) with [Ambassador Ingress](https://docs.seldon.io/projects/seldon-core/en/latest/examples/seldon_core_setup.html#Ambassador) and [Install Seldon Core](https://docs.seldon.io/projects/seldon-core/en/latest/examples/seldon_core_setup.html#Install-Seldon-Core). Instructions [also online](https://docs.seldon.io/projects/seldon-core/en/latest/examples/seldon_core_setup.html).

We will assume that ambassador (or Istio) ingress is port-forwarded to `localhost:8003`

## Setup MinIO

Use the provided [notebook](https://docs.seldon.io/projects/seldon-core/en/latest/examples/minio_setup.html) to install Minio in your cluster.
Instructions [also online](https://docs.seldon.io/projects/seldon-core/en/latest/examples/minio_setup.html).

We will assume that MinIO service is port-forwarded to `localhost:8090`

In [97]:
%%writefile rclone.conf
[s3]
type = s3
provider = minio
env_auth = false
access_key_id = "admin@seldon.io"
secret_access_key = "12341234"
endpoint = http://localhost:8090

Overwriting rclone.conf


In [98]:
%%writefile secret.yaml
apiVersion: v1
kind: Secret
metadata:
  name: seldon-rclone-secret
type: Opaque
stringData:
  RCLONE_CONFIG_S3_TYPE: s3
  RCLONE_CONFIG_S3_PROVIDER: minio
  RCLONE_CONFIG_S3_ENV_AUTH: "false"
  RCLONE_CONFIG_S3_ACCESS_KEY_ID: "admin@seldon.io"
  RCLONE_CONFIG_S3_SECRET_ACCESS_KEY: "12341234"
  RCLONE_CONFIG_S3_ENDPOINT: http://minio.minio-system.svc.cluster.local:9000

Overwriting secret.yaml


In [99]:
!kubectl apply -f secret.yaml

secret/seldon-rclone-secret configured


# Add Triton cofniguration

In [100]:
%%writefile models_repository/mnist/config.pbtxt
name: "mnist"
platform: "tensorflow_savedmodel"
max_batch_size: 100
dynamic_batching { preferred_batch_size: [ 50 ]}
instance_group [ { count: 2 }]
input [
  {
    name: "conv2d_input"
    data_type: TYPE_FP32
    dims: [ 28, 28, 1 ]
  }
]
output [
  {
    name: "dense_1"
    data_type: TYPE_FP32
    dims: [ 10 ]
  }
]

Overwriting models_repository/mnist/config.pbtxt


# Copy model into S3

In [116]:
%%bash

export RCLONE_CONFIG_S3_TYPE=s3
export RCLONE_CONFIG_S3_PROVIDER=minio
export RCLONE_CONFIG_S3_ENV_AUTH="false"
export RCLONE_CONFIG_S3_ACCESS_KEY_ID="admin@seldon.io"
export RCLONE_CONFIG_S3_SECRET_ACCESS_KEY="12341234"
export RCLONE_CONFIG_S3_ENDPOINT="http://localhost:8090"
            
rclone copy -vvv models_repository s3:triton-models/mnist-model

2021/10/12 19:33:32 DEBUG : rclone: Version "v1.56.2" starting with parameters ["/home/rskolasinski/.asdf/installs/rclone/1.56.2/bin/rclone" "copy" "-vvv" "models_repository" "s3:triton-models/mnist-model"]
2021/10/12 19:33:32 DEBUG : Creating backend with remote "models_repository"
2021/10/12 19:33:32 DEBUG : Using config file from "/home/rskolasinski/.config/rclone/rclone.conf"
2021/10/12 19:33:32 DEBUG : fs cache: renaming cache item "models_repository" to be canonical "/home/rskolasinski/work/seldon-core/examples/mnist_triton_e2e/models_repository"
2021/10/12 19:33:32 DEBUG : Creating backend with remote "s3:triton-models/mnist-model"
2021/10/12 19:33:32 DEBUG : Setting type="s3" for "s3" from environment variable RCLONE_CONFIG_S3_TYPE
2021/10/12 19:33:32 DEBUG : Setting provider="minio" for "s3" from environment variable RCLONE_CONFIG_S3_PROVIDER
2021/10/12 19:33:32 DEBUG : Setting env_auth="false" for "s3" from environment variable RCLONE_CONFIG_S3_ENV_AUTH
2021/10/12 19:33:32 DE

In [136]:
%%writefile rclone.conf
[s3]
type = s3
provider = Minio
env_auth = false
access_key_id = admin@seldon.io
secret_access_key = 12341234
endpoint = http://localhost:8090

Overwriting rclone.conf


In [1]:
!rclone -vvv --config="rclone.conf" copy models_repository s3://triton-models/mnist-model

2021/10/13 11:24:29 DEBUG : rclone: Version "v1.56.2" starting with parameters ["/home/rskolasinski/.asdf/installs/rclone/1.56.2/bin/rclone" "-vvv" "--config=rclone.conf" "copy" "models_repository" "s3://triton-models/mnist-model"]
2021/10/13 11:24:29 DEBUG : Creating backend with remote "models_repository"
2021/10/13 11:24:29 DEBUG : Using config file from "/home/rskolasinski/work/seldon-core/examples/mnist_triton_e2e/rclone.conf"
2021/10/13 11:24:29 DEBUG : fs cache: renaming cache item "models_repository" to be canonical "/home/rskolasinski/work/seldon-core/examples/mnist_triton_e2e/models_repository"
2021/10/13 11:24:29 DEBUG : Creating backend with remote "s3://triton-models/mnist-model"
2021/10/13 11:24:29 DEBUG : fs cache: renaming cache item "s3://triton-models/mnist-model" to be canonical "s3:triton-models/mnist-model"
2021/10/13 11:24:29 DEBUG : mnist/config.pbtxt: Size and modification time the same (differ by 0s, within tolerance 1ns)
2021/10/13 11:24:29 DEBUG : mnist/confi

In [139]:
!rclone --config="rclone.conf" ls s3:

    15048 triton-models/mnist-model/mnist/1/model.savedmodel/keras_metadata.pb
   157755 triton-models/mnist-model/mnist/1/model.savedmodel/saved_model.pb
  7216711 triton-models/mnist-model/mnist/1/model.savedmodel/variables/variables.data-00000-of-00001
     2303 triton-models/mnist-model/mnist/1/model.savedmodel/variables/variables.index
      321 triton-models/mnist-model/mnist/config.pbtxt


# Deploy and Test model

## Deploy

In [86]:
%%writefile deployment.yaml
apiVersion: machinelearning.seldon.io/v1
kind: SeldonDeployment
metadata:
  name: mnist
  namespace: seldon
spec:
  name: default
  predictors:
  - graph:
      implementation: TRITON_SERVER
      logger:
        mode: all
      modelUri: s3:triton-models/mnist-model
      envSecretRefName: seldon-rclone-secret
      name: mnist
      type: MODEL
    name: default
    replicas: 1
  protocol: kfserving

Overwriting deployment.yaml


In [87]:
!kubectl apply -f deployment.yaml

seldondeployment.machinelearning.seldon.io/mnist created


In [88]:
!kubectl rollout status deploy/$(kubectl get deploy -l seldon-deployment-id=mnist -o jsonpath='{.items[0].metadata.name}')

Waiting for deployment "mnist-default-0-mnist" rollout to finish: 0 of 1 updated replicas are available...
deployment "mnist-default-0-mnist" successfully rolled out


## Test

In [93]:
!curl -s http://localhost:8003/seldon/seldon/mnist/v2/models/mnist | jq .

{
  "name": "mnist",
  "versions": [
    "1"
  ],
  "platform": "tensorflow_savedmodel",
  "inputs": [
    {
      "name": "conv2d_input",
      "datatype": "FP32",
      "shape": [
        -1,
        28,
        28,
        1
      ]
    }
  ],
  "outputs": [
    {
      "name": "dense_1",
      "datatype": "FP32",
      "shape": [
        -1,
        10
      ]
    }
  ]
}


In [94]:
import numpy as np
import requests

In [95]:
URL = "http://localhost:8003/seldon/seldon/mnist"

data = {
    "inputs": [
        {
            "name": "conv2d_input",
            "data": np.random.rand(10, 28, 28, 1).tolist(),
            "datatype": "FP32",
            "shape": [10, 28, 28, 1],
        }
    ]
}

r = requests.post(f"{URL}/v2/models/mnist/infer", json=data)
predictions = np.array(r.json()["outputs"][0]["data"]).reshape(
    r.json()["outputs"][0]["shape"]
)
output = [np.argmax(x) for x in predictions]

In [96]:
output

[8, 8, 8, 8, 8, 9, 8, 8, 8, 8]